# Using Principal Component Analysis for Digital Signal Processing
### Ayush Chakraborty, Allyson Hur, Cherry Pham, Suki Sacks

## Abstract
Digital Signal Processing (DSP) involves manipulating and analyzing signals to extract meaningful information. DSP normally relies on predefined algorithms and mathematical models for signal-processing tasks. This project explores the integration of principal component analysis (PCA) into DSP to enhance its capabilities and adaptability.

The objective of this project is to investigate how PCA can be used for tasks such as signal denoising, modulation recognition, and feature extraction. By utilizing datasets containing diverse signal patterns, the project aims to develop a deeper understanding of the adaptability and versatility of PCA in regards to various signal processing challenges.


In [ ]:
message_txt = "Hello, world."
message_bits, message_bits_txt = generate_signal(message_txt)